<a href="https://colab.research.google.com/github/GurmitSeera/SQL-Exercises/blob/main/SQL_Excercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install postgresql server
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!sudo service postgresql start

# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"

# Setup a database with name `tfio_demo` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS testdb;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE testdb;'
!pip install psycopg2

In [ ]:
import pandas as pd
import psycopg2

In [3]:
connection_string = {'host':'localhost',
                     'dbname':'testdb',
                     'user':'postgres',
                     'password':'postgres',
                     'port':5432}
connection = psycopg2.connect(**connection_string)

# Proble Statement 1

## Setup

In [4]:
sql_query = """
create table salary(
  employee_id varchar(10),
  department_id varchar(10),
  salary double precision
);
"""
with connection.cursor() as cur:
  cur.execute(sql_query)
insert_query = """
insert into salary(employee_id, department_id, salary) 
values 
('1','1',100.0),
('2','1',200.0),
('3','1',300.0),
('4','2',400.0),
('5','2',600.0),
('6','2',100.0),
('7','3',500.0),
('8','3',500.0),
('9','4',1000.0)
;

"""
with connection.cursor() as cur:
  cur.execute(insert_query)

## Question 1
Show department_id, max_salary

In [5]:
department_max_salary = """ Select department_id, max(salary) as max_salary from salary group by department_id;
"""
pd.read_sql_query(department_max_salary, connection)

,department_id,max_salary
0,2,600.0
1,4,1000.0
2,3,500.0
3,1,300.0


## Question 2
Show department_id, employee_id, max_salary

In [6]:
query = """select employee_id, department_id, salary from (Select employee_id, department_id, salary,
 rank () over (partition by department_id order by salary desc, employee_id) as s_rank
from salary) as A where s_rank = 1;
"""
pd.read_sql_query(query,connection)

,employee_id,department_id,salary
0,3,1,300.0
1,5,2,600.0
2,7,3,500.0
3,9,4,1000.0


## Question 3
Create employee_info table with columns employee_id, employee_name, city

In [7]:

query_create_table2 = """ Create table employee_info(employee_id varchar(10),
employee_name varchar(64), city varchar(64)
);
"""
with connection.cursor() as cur:
  cur.execute(query_create_table2)

In [8]:
insert_query1 = """ Insert into employee_info (employee_id, employee_name, city)
values
('1', 'Ramesh', 'Delhi'),
('2', 'Rahul', 'Gurugram'),
('3', 'Sandeep', 'Kolkata'),
('4', 'Deepanshu', 'Sonipat'),
('5', 'Sapna', 'Noida'),
('6', 'Varun', 'Noida'),
('7', 'Himanshu', 'Bhopal'),
('8', 'Ashwini', 'Jaipur'),
('9', 'Vipul','Panipat')
;
"""
with connection.cursor() as cur:
  cur.execute(insert_query1)


## Question 4
Show departmentid, employeeid, employeename, max salary

In [9]:
query1 = """ select employee_id, department_id, employee_name, salary from (Select b.employee_id, b.department_id, b.salary, c.employee_name,
 rank () over (partition by b.department_id order by b.salary desc, b.employee_id) as s_rank
from salary b left join employee_info c on b.employee_id = c.employee_id) as A where s_rank = 1;
"""
pd.read_sql_query(query1,connection)

,employee_id,department_id,employee_name,salary
0,3,1,Sandeep,300.0
1,5,2,Sapna,600.0
2,7,3,Himanshu,500.0
3,9,4,Vipul,1000.0


## Question 5
Show employeeid, employeename, salary, salary %

In [10]:
## get employeeid, employeename, salary, salary %
query2 = """ Select employee_id, employee_name, salary,
100.0*coalesce(salary,0)/sum(coalesce(salary,0)) over () as salary_percentage from (
Select s.employee_id, e.employee_name, s.salary
from salary s left join employee_info e on s.employee_id = e.employee_id) as A ;
"""
pd.read_sql_query(query2, connection)

,employee_id,employee_name,salary,salary_percentage
0,1,Ramesh,100.0,2.702703
1,2,Rahul,200.0,5.405405
2,3,Sandeep,300.0,8.108108
3,4,Deepanshu,400.0,10.810811
4,5,Sapna,600.0,16.216216
5,6,Varun,100.0,2.702703
6,7,Himanshu,500.0,13.513514
7,8,Ashwini,500.0,13.513514
8,9,Vipul,1000.0,27.027027


## Question 6
Show employeeid, departmentid, salary where salary > 500.0

In [11]:
## get employeeid, departmentid, salary where salary > 500.0
query3 = """ Select employee_id, department_id, salary from salary where salary > 500.0;
"""
pd.read_sql_query(query3,connection)

,employee_id,department_id,salary
0,5,2,600.0
1,9,4,1000.0


## Question 7
create new column in salary table having age of employee with (1,30), (2,30), (3,31), (4,31), (5,32) , (6,33), (7,32), (8,33), (9,31)

In [12]:
Query4 = """ Begin transaction;
Alter table salary add age float(32) default 1.0 ;
Update salary set age = 30 where employee_id = '1';
Update salary set age = 30 where employee_id = '2';
Update salary set age = 31 where employee_id = '3';
Update salary set age = 31 where employee_id = '4';
Update salary set age = 32 where employee_id = '5';
Update salary set age = 33 where employee_id = '6';
Update salary set age = 32 where employee_id = '7';
Update salary set age = 33 where employee_id = '8';
Update salary set age = 31 where employee_id = '9';
END transaction;
"""
with connection.cursor() as cur:
  cur.execute(Query4)


## Question 8
Show age, average salary for each age

In [13]:
query = """ Select age, avg(salary) as avg_salary_for_each_age from salary group by age;
"""
pd.read_sql_query(query, connection)

,age,avg_salary_for_each_age
0,33.0,300.000000
1,31.0,566.666667
2,30.0,150.000000
3,32.0,550.000000


## Question 9
Show department id, age, average salary for each age and department

In [14]:
query = """ Select department_id, age, avg(salary) as avg_salary_for_each_age from salary group by 1,2;
"""
pd.read_sql_query(query, connection)

,department_id,age,avg_salary_for_each_age
0,2,33.0,100.0
1,3,33.0,500.0
2,3,32.0,500.0
3,2,31.0,400.0
4,4,31.0,1000.0
5,1,31.0,300.0
6,2,32.0,600.0
7,1,30.0,150.0


## Question 10
update employee id 1 salary to 1000.0 from 100.0

In [15]:
query = """ Update salary set salary = 100.0 where employee_id = '1';
"""
with connection.cursor() as cur:
  cur.execute(query)

## Question 11
show table with employee id, employee name, city where salary > 500.0

In [16]:
query = """ Select s.employee_id, e.employee_name, e.city, s.salary
from salary s left join employee_info e on s.employee_id = e.employee_id where s.salary > 500.0;
"""
pd.read_sql_query(query, connection)

,employee_id,employee_name,city,salary
0,5,Sapna,Noida,600.0
1,9,Vipul,Panipat,1000.0


# Problem Statement 2

In [19]:
Sql_query = """
CREATE TABLE Warehouses (
   Code INTEGER NOT NULL,
   Location VARCHAR(255) NOT NULL ,
   Capacity INTEGER NOT NULL,
   PRIMARY KEY (Code)
 );
CREATE TABLE Boxes (
    Code CHAR(4) NOT NULL,
    Contents VARCHAR(255) NOT NULL ,
    Value REAL NOT NULL ,
    Warehouse INTEGER NOT NULL,
    PRIMARY KEY (Code),
    FOREIGN KEY (Warehouse) REFERENCES Warehouses(Code)
 ) ;
"""
with connection.cursor() as cur:
  cur.execute(Sql_query)
insert_query = """INSERT INTO Warehouses(Code,Location,Capacity)
                  values
                  (1,'Chicago',3),
                  (2,'Chicago',4),
                  (3,'New York',7),
                  (4,'Los Angeles',2),
                  (5,'San Francisco',8)
                  ;

                  INSERT INTO Boxes(Code,Contents,Value,Warehouse) 
                  VALUES
                  ('0MN7','Rocks',180,3),
                  ('4H8P','Rocks',250,1),
                  ('4RT3','Scissors',190,4),
                  ('7G3H','Rocks',200,1),
                  ('8JN6','Papers',75,1),
                  ('8Y6U','Papers',50,3),
                  ('9J6F','Papers',175,2),
                  ('LL08','Rocks',140,4),
                  ('P0H6','Scissors',125,1),
                  ('P2T6','Scissors',150,2),
                  ('TU55','Papers',90,5)
                  ;
"""

# Question 1
Select the code of each box, along with the name of the city the box is located in.

In [21]:
query= """ SELECT Boxes.Code, Location
           FROM Warehouses 
           INNER JOIN Boxes
           ON Warehouses.Code = Boxes.Warehouse;
"""
pd.read_sql_query = (query, connection)